In [1]:
import nltk
# Punkt permite separar un texto en frases.
nltk.download('punkt')
# Descarga todas las palabras vacias, es decir, aquellas que no aportan nada al significado del texto
nltk.download('stopwords')
#Paquete WordNetLemmatizer, este es usado para encontrar el lema de cada palabr
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jafie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jafie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jafie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jafie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
# Librerias
import pandas as pd
import numpy as np
import sys
import nltk

!{sys.executable} -m pip install pandas-profiling
!pip install vaderSentiment
!pip install seaborn
!pip install imbalanced-learn
!pip install nltk
!pip install scipy
from imblearn.pipeline import Pipeline as ImbPipeline
import nltk
nltk.data.path.append("C:/Users/Felipe Jaimes/nltk_data")
nltk.download('punkt', force=True)
import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from datetime import datetime
from sklearn.model_selection import cross_val_score
from scipy import stats as st

from sklearn.base import BaseEstimator, ClassifierMixin
import seaborn as sns
from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from sklearn.compose import ColumnTransformer

  Using cached joblib-1.1.1-py2.py3-none-any.whl.metadata (5.2 kB)
Using cached joblib-1.1.1-py2.py3-none-any.whl (309 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Successfully uninstalled joblib-1.4.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-learn 1.5.1 requires joblib>=1.2.0, but you have joblib 1.1.1 which is incompatible.


  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.4.2 which is incompatible.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jafie\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [3]:
data_cargada=pd.read_csv('Data/fake_news_spanish.csv', sep=';', encoding = 'utf-8')
# Asignación a una nueva variable de los datos leidos
data=data_cargada

In [4]:
data

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018
...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022


In [5]:
# data['Titulo'] = data['Titulo'].astype('string')
# data['Descripcion'] = data['Descripcion'].astype('string')

# data['Fecha'] = pd.to_datetime(data['Fecha'], format='%d/%m/%Y')
# df=data
from sklearn.base import BaseEstimator, TransformerMixin


class TypeAndDatePreprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['Titulo'] = X['Titulo'].astype('string')
        X['Descripcion'] = X['Descripcion'].astype('string')
        return X


In [6]:
# def imputar_titulo_desde_descripcion(titulo, descripcion, max_palabras=5):
#     if pd.isna(titulo):  
#         if isinstance(descripcion, str):  
#             palabras = descripcion.split()  
#             return ' '.join(palabras[:max_palabras])  
#         return ''  
#     return titulo  

# print(f"Filas con Título nulo antes: {df['Titulo'].isna().sum()}")
# df['Titulo'] = df.apply(lambda row: imputar_titulo_desde_descripcion(row['Titulo'], row['Descripcion']), axis=1)
# print(f"Filas con Título nulo después: {df['Titulo'].isna().sum()}")


class ImputarTituloDesdeDescripcion(BaseEstimator, TransformerMixin):
    def __init__(self, max_palabras=5):
        self.max_palabras = max_palabras

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        def imputar_titulo(titulo, descripcion):
            if pd.isna(titulo):
                if isinstance(descripcion, str):
                    palabras = descripcion.split()
                    return ' '.join(palabras[:self.max_palabras])
                return ''
            return titulo

        X['Titulo'] = X.apply(lambda row: imputar_titulo(row['Titulo'], row['Descripcion']), axis=1)
        return X


In [7]:
# def preprocess_text(text):
#     if not isinstance(text, str):
#         return ''
#     text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')    
#     text = text.lower()
#     tokens = word_tokenize(text)
#     tokens = [re.sub(r'[^\w\s]', '', word) for word in tokens if word.isalnum()]    
#     tokens = [p.number_to_words(word) if word.isdigit() else word for word in tokens]
#     # tokens = [word for word in tokens if word not in stop_words]
#     # tokens = [lemmatizer.lemmatize(word) for word in tokens]
#     return ' '.join(tokens)  


# df['Titulo'].apply(lambda x: preprocess_text(x))
# df['Descripcion'].apply(lambda x: preprocess_text(x))


class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, columnas):
        self.columnas = columnas
        self.inflector = inflect.engine()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.columnas:
            X[col] = X[col].apply(self._preprocess_text)
        return X

    def _preprocess_text(self, text):
        if not isinstance(text, str):
            return ''
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')    
        text = text.lower()
        tokens = word_tokenize(text)
        tokens = [re.sub(r'[^\w\s]', '', word) for word in tokens if word.isalnum()]
        tokens = [self.inflector.number_to_words(word) if word.isdigit() else word for word in tokens]
        return ' '.join(tokens)




In [8]:
class CombinarTexto(BaseEstimator, TransformerMixin):
    def __init__(self, col1='Titulo', col2='Descripcion'):
        self.col1 = col1
        self.col2 = col2

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['combined_text'] = (X[self.col1].fillna('') + ' ' + X[self.col2].fillna('')).str.strip()

        result = X['combined_text'].astype(str).values
        
        return result

In [9]:
pipelineSelectedModel = Pipeline([
    ('type_date', TypeAndDatePreprocessor()),
    ('imputar_titulo', ImputarTituloDesdeDescripcion(max_palabras=5)),
    ('text_preprocessing', TextPreprocessor(columnas=['Titulo', 'Descripcion'])),
    ('combinar_texto', CombinarTexto()), 
    ('vectorizer', TfidfVectorizer(max_features=5000)),
    ('model', RandomForestClassifier(
        n_estimators=100, 
        min_samples_split=40,  
        max_leaf_nodes=500, 
        class_weight="balanced", 
        random_state=42
    ))
])

In [10]:
X = data[['Titulo', 'Descripcion']] 
y = data['Label'] 

In [11]:
print(X.shape, y.shape)

(57063, 2) (57063,)


In [12]:
pipelineSelectedModel.fit(X, y)

Pipeline(steps=[('type_date', TypeAndDatePreprocessor()),
                ('imputar_titulo', ImputarTituloDesdeDescripcion()),
                ('text_preprocessing',
                 TextPreprocessor(columnas=['Titulo', 'Descripcion'])),
                ('combinar_texto', CombinarTexto()),
                ('vectorizer', TfidfVectorizer(max_features=5000)),
                ('model',
                 RandomForestClassifier(class_weight='balanced',
                                        max_leaf_nodes=500,
                                        min_samples_split=40,
                                        random_state=42))])

In [ ]:
import joblib
from joblib import dump

filename = 'api/modelNewPipe.joblib'
dump(pipelineSelectedModel, filename)

['api/modelNewPipe.joblib']

: 